## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # Functions without parameters such as relu, tanh, ...
from torch.utils.data import DataLoader  # Dataset management. It helps us create mini-batches to train and ..
import torchvision.datasets as datasets  # Datasets such as MNIST
import torchvision.transforms as transforms  # transformations that we can do on our datasets 

## Create Fully Connected Network

In [2]:
class NN(nn.Module):
    """
    input_size: 28x28 since MNIST dataset has 28x28 images. So we have 784 neurons
    """
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()  # run the initialization of nn.Module
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x        

## Set device

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyperparameters

In [4]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

## Load data

In [5]:
# Train data
train_dataset = datasets.MNIST(root='dataset/',
                train=True,
                transform=transforms.ToTensor(), # The data is numpy and we want it to be transformed to PyTorch Tensors
                download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Test data
test_dataset = datasets.MNIST(root='dataset/',
                train=False,
                transform=transforms.ToTensor(), # The data is numpy and we want it to be transformed to PyTorch Tensors
                download=True)
test_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

C:\Users\Asus\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Initialize network

In [6]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

## Loss and optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Train Network

In [11]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to cuda if possible
        data = data.to(device=device)  # data size is 64x1x28x28. Since we have 64 image in batch size having 1 channel each.
        targets = targets.to(device=device)  # 64 elements since we have 64 labels showing which number it is
        
        # get the correct shape
        data = data.reshape(data.shape[0], -1)  # flatten 64x1x28x28 to 64x784
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()  # set all the gradients to zero for each batch
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step() 

In [12]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
        
    num_correct = 0
    num_samples = 0
    model.eval()  # Turning off dropout, BatchNorms, etc during model evaluation
    
    with torch.no_grad(): # calculating gradients while computing accuracy is not necessary
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)  # 64x10, we only want to get the index of max in the second dimension
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0) # which is 64 in our case
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}")
        
        
    model.train()  # Turn dropout, BatchNorms, etc back 

In [13]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 56072 / 60000 with accuracy 93.45
Checking accuracy on training data
Got 56072 / 60000 with accuracy 93.45
